# Iris Dataset Clustering

## Employ multiple clustering techniques on the Iris dataset, with and without PCA.  Evaluate results.

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from pandas_profiling import ProfileReport
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans, MeanShift
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from sklearn.cluster import DBSCAN
import plotly.express as px

## Define Function for Clustering

In [ ]:
def iris_clustering(func, data, target, **args):
        
        """This function generates a clustering model using a scikit-learn function, displays a confusion matrix and plot of
        the data (using the first 3 columns of the data), and returns the model
            
            Arguments:
            func - A scikit-learn clustering function
            data - A pandas dataframe containing the features to be used in clustering
            target - The target values for comparison with the model predictions
            
            Any keyword arguments requried for the specific clustering function should also be included at the end of the call
        """
        
        normalized = normalize(data)
        
        model = func(**args)
        preds = model.fit_predict(normalized)
        
        preds_df = data.copy()
        preds_df['target'] = target
        preds_df['preds'] = preds
        
        conf = pd.crosstab(preds_df['preds'], preds_df['target'], margins=True)
        print("Confusion Matrix:")
        display(conf)
        
        homogeneity = metrics.homogeneity_score(preds_df["target"], preds_df["preds"])
        completeness = metrics.completeness_score(preds_df["target"], preds_df["preds"])
        v_measure = metrics.v_measure_score(preds_df["target"], preds_df["preds"], beta=1.0)

        print(f"Homogeneity_score: {homogeneity:.2f}, measures that each cluster contains only members of a single class")
        print(f"Completeness_score: {completeness:.2f}, easures that all members of a given class are assigned to the same cluster")
        print(f"V-measure: {v_measure:.2f}, harmonic mean of homogeneity and completeness")

        scatter = px.scatter_3d(preds_df, x=preds_df.columns[0], y=preds_df.columns[1], z=preds_df.columns[2],
                            color = 'target', symbol = 'preds')
        scatter.show()
        
        return model
    

## Load Dataset, Explore and Display Features

In [ ]:
iris = load_iris()
iris_df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
iris_df['target'] = iris_df['target'].replace([0,1,2],['setosa', 'versicolor', 'virginica'])

* There are 3 unique target variables: setosa, versicolor and virginica; each a species of Iris
* Changed the target attribute labels to a descriptive string vs numerical category for ease in analysis

In [ ]:
iris_df

In [ ]:
iris_df.shape

In [ ]:
iris_df.info()

* The dataset contains 150 observations, has 4 predictive attributes and 1 target variable
* The 4 predictive attributes are numerical, the target variable is categorical

In [ ]:
iris_df['target'].describe()

In [ ]:
iris_df.describe()

* petal length has the largest range and greatest variation of the 4 attributes, and also has the greatest difference of the 4 between its mean and median
* for the other three attributes, their mean approximates their median which suggests the mean is not affected by outliers

In [ ]:
profile = ProfileReport(iris_df)
profile

In [ ]:
iris_df.corr()

### Observations:

* The dataset has zero missing observations
* This is a balanced dataset in that each of the three target labels have the same number of observations
* The distributions of sepal length and sepal width are fairly normal
* The distributions of petal length and petal width both have two distinct groupings
* Correlation - because the 4 predictive attributes are all numerical, refer to the Pearson's r chart, above:
    * Sepal width and sepal length appear to be uncorrelated
    * Petal width and petal length appear to be highly correlated 
    * Petal length and sepal length appear to be fairly correlated
    * Petal width and sepal length also appear to be correlated, though less so than petal length and sepal length    
* Correlation - see pair plot graphs below for visual confirmation of the above correlation observations

## Build Elbow Plot to determine optimal number of clusters

In [ ]:
# Create Iris data frame without the target column

iris_features = iris_df.drop(columns='target')
iris_target = iris_df['target']
iris_features.head()

In [ ]:
# Create Elbow plot of inertia values to determine optimal number of clusters to use in a K-Means clustering method
# inertia is the sum of the squared distances of observations to their closest cluster center

inertia_values = []
cluster_centers = []
K = range(1,11) #Try number of clusters from 1 to 10

for k in K:
    k_mean_model = KMeans(n_clusters = k)
    k_mean_model.fit(iris_features)
    inertia_values.append(k_mean_model.inertia_) #track the inertia values for each number of clusters
    cluster_centers.append(k_mean_model.cluster_centers_) #track the cluster centers for each number of clusters

# Create data frame of values for elbow plot
elbow_data = {'Number of Clusters': K, 'Inertia': inertia_values}
elbow_df = pd.DataFrame(elbow_data) 

In [ ]:
# Graph the Elbow plot

fig_dims = (6, 3)
fig, ax = plt.subplots(figsize=fig_dims)

sns.set_theme(style = "whitegrid")
sns.pointplot(data = elbow_df, x = 'Number of Clusters' ,y = 'Inertia', markers=["o"])\
.set(title='Eblow Plot using Inertia for K-Means Clustering of Iris Data');

The optimal number of clusters is at the "elbow" of the graph, where the Inertia begins to decline in a linear fashion.  In this case the optimal number is 3.

In [ ]:
# Display cluster centers for k = 3 clusters in k-means model
cluster_centers[2:3]

## Reduce data frame using Principal Components Analysis

In [ ]:
iris_features

In [ ]:
# PCA is affected by scale, so we need to scale our features before applying PCA
# StandardScaler will standardize the dataset’s features onto unit scale (mean = 0 and variance = 1) 

x = iris_features.values
scaled_array = StandardScaler().fit_transform(x) #This is an array of the standardized values of the four feature columns
iris_standardized = pd.DataFrame(data= np.c_[scaled_array], \
                             columns = ('sepal length', 'sepal width', 'petal length', 'petal width'))

# View standardized data frame
iris_standardized.head()

In [ ]:
# The first decision in PCA is to select the number of components to reduce to.
# The goal is to reduce dimensions while still retaining most of the variance of the features.

# Start with components=4 (use all) to assess what % of the variance each principal component contains
pca = PCA(n_components=4)
principalComponents = pca.fit_transform(scaled_array)

print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.cumsum())

In [ ]:
# Plotting the Explained variance as a function of Principal Components considered

figure=plt.figure(figsize=(9,5))
plt.plot(range(1,5),pca.explained_variance_ratio_.cumsum(),marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Scree Plot')
plt.locator_params(axis='x', nbins=4)

plt.show()

Based on the cumulative % of variance explained by the principal components, I can see that principal components 1, 2, and 3 contain 99.4% of the variation (information). We can also see on the Scree Plot above that the explained variance gain from 2 to 3 is signficant. As such 3 principal compnents will be used for the PCA data frame in this analysis.

In [ ]:
# Apply PCA to the scaled version of the Iris features, using 3 principal components (based on above analysis)
# Create and display PCA Features data frame. 
# Note: there isn't particular meaning assigned to each principal component, the new components are just the two main 
# dimensions of variation - which are linear combinations of the original variables.

iris_model = PCA(n_components=3)
principalComponents = iris_model.fit_transform(scaled_array)
iris_PCA = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])
iris_PCA.head()

In [ ]:
# Scatterplot of the iris data reduced to 3 Principal Components
iris_PCA_target=iris_PCA.assign(target=iris_df['target'])
fig = px.scatter_3d(iris_PCA_target, x='principal component 1', y='principal component 2', z='principal component 3',
              color='target', size_max=8, opacity = .7)
fig.show()

## Employ Clustering Techniques - on original data frame and on PCA data frame
* Partitioning Method: K-Means
* Hierarchical Method: sklearn's AgglomerativeClustering
* Density-based Method: DBSCAN

### Partitioning Method: K-Means

#### K-Means on original Iris data set

In [ ]:
iris_features

In [ ]:
iris_features = iris_df.drop(columns='target')
iris_features.head()

In [ ]:
# Run KMeans on the original features of the Iris data frame, using 3 clusters

k_mean_model = KMeans(n_clusters = 3)
k_mean_model.fit(iris_features)

# Create new Datafram with predicted cluster from KMeans and target label from iris_df
iris_kmeans=iris_features.assign(k_means=k_mean_model.predict(iris_features),
                                 target=iris_df['target'])

In [ ]:
# Display modified iris_features data frame
iris_kmeans.head()

In [ ]:
# Create table to assess clustering accuracy

my_crosstab = pd.crosstab(index=iris_kmeans["k_means"],columns=iris_kmeans["target"], margins=True) 
my_crosstab

The above table indicates an purity score of 89.3% ((36+50+48)/150 = .89333)

In [ ]:
# Clustering performance evaluation - additional measures

# Rand Index - Adjusted: a function that measures the similarity of the two assignments, adjusted corrects for chance
print('Adjusted Rand Index:', metrics.adjusted_rand_score(iris_kmeans["target"], iris_kmeans["k_means"]),\
     ", measures the similarity of the two assignments: true label and predicted cluster",'\n')
#homogeneity_score: each cluster contains only members of a single class.
print('Homogeneity_score:',metrics.homogeneity_score(iris_kmeans["target"], iris_kmeans["k_means"]),\
     ", measures that each cluster contains only members of a single class",'\n')
#completeness_score: all members of a given class are assigned to the same cluster.
print('Completeness_score:', metrics.completeness_score(iris_kmeans["target"], iris_kmeans["k_means"]),\
     ", measures that all members of a given class are assigned to the same cluster",'\n')
#v_measure_score: harmonic mean of homogeneity and completeness).
print('V-measure:', metrics.v_measure_score(iris_kmeans["target"], iris_kmeans["k_means"], beta=1.0),\
     ", harmonic mean of homogeneity and completeness, beta>1 gives more weight to homogeneity")

In [ ]:
# Graph actual labels vs. clustering for sepal length/sepal width and petal length/petal width

fig, ((ax1, ax2)) = plt.subplots(2, 1, figsize = (15, 20))
sns.scatterplot(x='sepal length (cm)', y='sepal width (cm)', data = iris_kmeans, hue='k_means',
                style='target',palette="deep",ax=ax1, s=100)\
.set(title='Sepal Length vs. Width:     Color=K-Means, Shape=Actual Target') 
sns.scatterplot(x='petal length (cm)', y='petal width (cm)', data = iris_kmeans, hue='k_means',
                style='target',palette="deep",ax=ax2, s=100)\
.set(title='Petal Length vs. Width:     Color=K-Means, Shape=Actual Target');

#### K=Means on PCA Features data frame

In [ ]:
# Recall the structure of the PCA Features data frame

iris_PCA.head()

In [ ]:
# Run KMeans on the PCA Features data frame, using 3 clusters

k_mean_model = KMeans(n_clusters = 3)
k_mean_model.fit(iris_PCA)

# Add column to iris_PCA_df dataframe: predicted cluster from KMeans
iris_PCA_kmeans = iris_PCA.assign(k_means=k_mean_model.predict(iris_PCA),
                                  target=iris_df['target'])

In [ ]:
# Display modified iris_PCA_df data frame
iris_PCA_kmeans.head()

In [ ]:
# Create table to assess clustering accuracy

my_crosstab = pd.crosstab(index=iris_PCA_kmeans["k_means"],columns=iris_PCA_kmeans["target"], margins=True) 
my_crosstab

The above table indicates an purity score of 83.3% ((36+50+39)/150 = .8333)

In [ ]:
# Clustering performance evaluation - additional measures

# Rand Index - Adjusted: a function that measures the similarity of the two assignments, adjusted corrects for chance
print('Adjusted Rand Index: ', metrics.adjusted_rand_score(iris_PCA_kmeans["target"], iris_PCA_kmeans["k_means"]),\
     ", measure the similarity of the two assignments: true label and predicted cluster",'\n')
#homogeneity_score: each cluster contains only members of a single class.
print('Homogeneity_score: ',metrics.homogeneity_score(iris_PCA_kmeans["target"], iris_PCA_kmeans["k_means"]),\
     ", measures that each cluster contains only members of a single class",'\n')
#completeness_score: all members of a given class are assigned to the same cluster.
print('Completeness_score: ', metrics.completeness_score(iris_PCA_kmeans["target"], iris_PCA_kmeans["k_means"]),\
     ", measures that all members of a given class are assigned to the same cluster",'\n')
#v_measure_score: harmonic mean of homogeneity and completeness).
print('V-measure: ', metrics.v_measure_score(iris_PCA_kmeans["target"], iris_PCA_kmeans["k_means"], beta=1.0),\
     ", harmonic mean of homogeneity and completeness, beta>1 gives more weight to homogeneity")

In [ ]:
# Graph actual labels vs. clustering for sepal length/sepal width and petal length/petal width

fig, ((ax1)) = plt.subplots(1, figsize = (15, 8))
sns.scatterplot(x='principal component 1', y='principal component 2', data = iris_PCA_kmeans,
                hue='k_means',style='target', palette="deep",s=100, ax=ax1)\
.set(title='Principal Component 1 vs. Principal Component 2: Color=K-Means, Shape/Size=Actual Target');

## Hierarchical Method: sklearn's AgglomerativeClustering

In [ ]:
iris_features.head()

In [ ]:
iris_features[iris_features.columns[:-2]].head()

In [ ]:
iris_normalized = pd.DataFrame(normalize(iris_features),columns=iris_features.columns[:])
iris_normalized.head()

In [ ]:
# Like an elbow plot for k-means, a dendogram helps determine the ideal number of clusters for hierarchical clustering
# Note: although this chart shows 2 clusters at the second level, we'll go down to the 3rd level and use 3 clusters

plt.figure(figsize=(18, 7))  
plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(iris_normalized, method='ward'))
plt.axhline(y=0.5, color='r', linestyle='--');

In [ ]:
# Note: the linkage criterion determines which distance to use between sets of observation.
# ‘ward’ minimizes the variance of the clusters being merged, euclidean is the distance used for this criterion

cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')  
cluster.fit_predict(iris_normalized)

In [ ]:
# Creating new data frame using Agglomerative Clustering
iris_agglo=iris_normalized.assign(hierarchical=cluster.fit_predict(iris_normalized),
                                 target=iris_df['target'])
iris_agglo.head()

In [ ]:
# Create table to assess clustering accuracy

my_crosstab = pd.crosstab(index=iris_agglo['hierarchical'],columns=iris_agglo["target"], margins=True) 
my_crosstab

The above table indicates an purity score of 96.0% ((48+50+46)/150 = 0.96)

In [ ]:
# Clustering performance evaluation - additional measures

# Rand Index - Adjusted: a function that measures the similarity of the two assignments, adjusted corrects for chance
print('Adjusted Rand Index: ', metrics.adjusted_rand_score(iris_agglo["target"], iris_agglo['hierarchical']),\
     ", measure the similarity of the two assignments: true label and predicted cluster",'\n')
#homogeneity_score: each cluster contains only members of a single class.
print('Homogeneity_score: ',metrics.homogeneity_score(iris_agglo["target"], iris_agglo['hierarchical']),\
     ", measures that each cluster contains only members of a single class",'\n')
#completeness_score: all members of a given class are assigned to the same cluster.
print('Completeness_score: ', metrics.completeness_score(iris_agglo["target"], iris_agglo['hierarchical']),\
     ", measures that all members of a given class are assigned to the same cluster",'\n')
#v_measure_score: harmonic mean of homogeneity and completeness).
print('V-measure: ', metrics.v_measure_score(iris_agglo["target"], iris_agglo['hierarchical'], beta=1.0),\
     ", harmonic mean of homogeneity and completeness, beta>1 gives more weight to homogeneity",'\n')

In [ ]:
# Graph actual labels vs. clustering for sepal length/sepal width and petal length/petal width

fig, ((ax1, ax2)) = plt.subplots(2, 1, figsize = (15, 15))
sns.scatterplot(x='sepal length (cm)', y='sepal width (cm)', data = iris_agglo, hue='hierarchical',style='target',size='target', palette="deep",ax=ax1)\
.set(title='Sepal Length vs. Width: Color=hierarchical cluster, Shape/Size=Actual Target')
sns.scatterplot(x='petal length (cm)', y='petal width (cm)', data = iris_agglo, hue='hierarchical',style='target',size='target',palette="deep",ax=ax2)\
.set(title='Petal Length vs. Width: Color=hierarchical cluster, Shape/Size=Actual Target');# Scatterplot of the iris data reduced to 3 Principal Components

# Creating 3D axes
# fig = plt.figure(figsize=(6,6))
# ax = Axes3D(fig, auto_add_to_figure=False)
# fig.add_axes(ax)

# cmap = ListedColormap(sns.color_palette("husl", 256).as_hex()) # Colormap magic

# scatter_3d = ax.scatter(pca_3_df['PC1'], pca_3_df['PC2'], pca_3_df['PC3'], c=colors,
#                          marker='o', cmap=cmap, alpha=1)

# ax.set_xlabel('PC1')
# ax.set_ylabel('PC2')
# ax.set_zlabel('PC3')
# legend = ax.legend(*scatter_3d.legend_elements(), title='Class')
# ax.add_artist(legend)
# plt.show()

fig2 = px.scatter_3d(x=iris_PCA['principal component 1'], y=iris_PCA['principal component 2'], 
                     z=iris_PCA['principal component 3'], color=iris_df['target'])
fig2.show()

### Density-based Method: DBSCAN

The scikit-learn implementation provides a default for the eps and min_samples parameters, but you’re generally expected to tune those. The eps parameter is the maximum distance between two data points to be considered in the same neighborhood. The min_samples parameter is the minimum amount of data points in a neighborhood to be considered a cluster.

In [ ]:
iris_features

In [ ]:
x = iris_features.copy()
x

In [ ]:
# Check through this example - need to standardize? normalize? Look up diff between the two and when they should be used.
# Do for original df and for pca separately
# eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other
# min_samples: The number of samples (or total weight) in a neighborhood for a point to be considered as a core point
# noise: points that don't have the min # of points within the eps distance (not core points)
dbscan=DBSCAN(eps = 0.45, min_samples = 5)
dbscan.fit(x)
pca=PCA(n_components=2).fit(x)
pca_2d=pca.transform(x)

for i in range(0, pca_2d.shape[0]):
    if dbscan.labels_[i] == 0:
        c1 = plt.scatter(pca_2d[i, 0], pca_2d[i, 1], c='r', marker='+')
    elif dbscan.labels_[i] == 1:
        c2 = plt.scatter(pca_2d[i, 0], pca_2d[i, 1], c='g', marker='o')
    elif dbscan.labels_[i] == -1:
        c3 = plt.scatter(pca_2d[i, 0], pca_2d[i, 1], c='b', marker='*')
        
plt.legend([c1, c2, c3], ['Cluster 1', 'Cluster 2', 'Noise'])
plt.title('DBSCAN finds 2 clusters and Noise')
plt.show()

In [ ]:
# Check through this example - need to standardize? normalize? Look up diff between the two and when they should be used.
# Do for original df and for pca separately
# eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other
# min_samples: The number of samples (or total weight) in a neighborhood for a point to be considered as a core point
# noise: points that don't have the min # of points within the eps distance (not core points)
dbscan=DBSCAN(eps = 0.45, min_samples = 5)
dbscan.fit(x)

In [ ]:
# Add columns to iris_features dataframe: predicted cluster from KMeans and target label from iris_df
x['DBSCAN_cluster']=dbscan.labels_
x['target']=iris_df['target']
x

In [ ]:
# Create table to assess clustering accuracy

my_crosstab = pd.crosstab(index=x['DBSCAN_cluster'],columns=x['target'], margins=True) 
my_crosstab

The above table indicates an purity score of 84.0% ((48+43+35)/150 = 0.84)- NOT SURE THIS CALC APPLIES HERE B/C it only produced two clusters.

In [ ]:
# Clustering performance evaluation - additional measures

# Rand Index - Adjusted: a function that measures the similarity of the two assignments, adjusted corrects for chance
print('Adjusted Rand Index: ', metrics.adjusted_rand_score(x["target"], x['DBSCAN_cluster']),\
     ", measure the similarity of the two assignments: true label and predicted cluster")
#homogeneity_score: each cluster contains only members of a single class.
print('Homogeneity_score: ',metrics.homogeneity_score(x["target"], x['DBSCAN_cluster']),\
     ", measures that each cluster contains only members of a single class")
#completeness_score: all members of a given class are assigned to the same cluster.
print('Completeness_score: ', metrics.completeness_score(x["target"], x['DBSCAN_cluster']),\
     ", measures that all members of a given class are assigned to the same cluster")
#v_measure_score: harmonic mean of homogeneity and completeness).
print('V-measure: ', metrics.v_measure_score(x["target"], x['DBSCAN_cluster'], beta=1.0),\
     ", harmonic mean of homogeneity and completeness, beta>1 gives more weight to homogeneity")

In [ ]:
# Graph actual labels vs. clustering for sepal length/sepal width and petal length/petal width

fig, ((ax1, ax2)) = plt.subplots(2, 1, figsize = (15, 20))
sns.scatterplot(x='sepal length (cm)', y='sepal width (cm)', data = x, hue='DBSCAN_cluster',
                style='target',s=100, palette="deep",ax=ax1)\
.set(title='Sepal Length vs. Width: Color=DBSCAN_cluster, Shape=Actual Target')
sns.scatterplot(x='petal length (cm)', y='petal width (cm)', data = x, hue='DBSCAN_cluster',
                style='target',s=100,palette="deep",ax=ax2)\
.set(title='Petal Length vs. Width: Color=DBSCAN_cluster, Shape/Size=Actual Target');

### Mean Shift Clustering

In [ ]:
iris_features

In [ ]:
# Run Mean shift clustering on the orgininal iris 
# data using an estimated bandwidth

mean_clusters = MeanShift(n_jobs=-1)
mean_clusters.fit(iris_features)

labels = mean_clusters.labels_
iris_meanshift=iris_features.assign(meanshift=labels,
                                 target=iris_df['target'])
iris_meanshift.head(3)

In [ ]:
my_crosstab = pd.crosstab(index=iris_meanshift["meanshift"],columns=iris_meanshift["target"], margins=True) 
my_crosstab

Meanshift clustering fails to determine there are 3 clusters for the original Iris data frame

In [ ]:
# Run Mean shift clustering on the PCA iris data using an estimated bandwidth

mean_clusters = MeanShift(n_jobs=-1)
mean_clusters.fit(iris_PCA)

labels2 = mean_clusters.labels_
iris_meanshift_PCA=iris_PCA.assign(meanshift=labels2,
                                 target=iris_df['target'])
iris_meanshift_PCA.head(3)

In [ ]:
my_crosstab = pd.crosstab(index=iris_meanshift_PCA["meanshift"],columns=iris_meanshift_PCA["target"], margins=True) 
my_crosstab

Meanshift clustering fails to determine there are 3 clusters for the PCA Iris data frame

## Implementation with clustering function (I didn't want to delete any previous work yet, but I think doing it this way could help clean things up)

## KMeans

In [ ]:
iris_clustering(KMeans, iris_features, iris_target, n_clusters=3)

In [ ]:
iris_clustering(KMeans, iris_PCA, iris_target, n_clusters=3)

## Agglomerative

In [ ]:
iris_clustering(AgglomerativeClustering, iris_features, iris_target, n_clusters=3, affinity='euclidean', linkage='ward')

In [ ]:
iris_clustering(AgglomerativeClustering, iris_PCA, iris_target, n_clusters=3, affinity='euclidean', linkage='ward')

## DBScan

In [ ]:
iris_clustering(DBSCAN, iris_features, iris_target, eps = 0.2, min_samples = 5)

In [ ]:
iris_clustering(DBSCAN, iris_PCA, iris_target, eps = 0.48, min_samples = 5)

## Mean Shift

In [ ]:
iris_clustering(MeanShift, iris_features, iris_target, n_jobs=-1)

In [ ]:
iris_clustering(MeanShift, iris_PCA, iris_target, n_jobs=-1)

## Conclusions: 